In [1]:
import os

def import_from_directory(directory, module_name):
    current_dir = os.getcwd()
    os.chdir(directory)
    module = __import__(module_name)
    os.chdir(current_dir)
    return module

test_base = import_from_directory('/home/jovyan/work/spectrum/notebooks/src', 'test_base')
model_base = import_from_directory('/home/jovyan/work/spectrum/notebooks/src', 'model_base')

2024-05-07 04:59:27.415654: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 04:59:27.453343: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 04:59:27.453382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 04:59:27.454558: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-07 04:59:27.461204: I tensorflow/core/platform/cpu_feature_guar

In [2]:
SEED = 42
model_base.set_seed(SEED)

desired_directory = '/home/jovyan/work/spectrum/notebooks/'
os.chdir(desired_directory)

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('Data/test_data/rikenlipid_data.csv')
df['datasetID'] = df['dataset'].str.split('_').str[0]
df_neg = df[df['Adducttype'].str.contains('\]\-')].reset_index(drop=True)

In [5]:
df_neg2 = model_base.cal_df_sel_column(df_neg).drop_duplicates().reset_index(drop=True)
df_neg_wide = model_base.cal_wide_df(df_neg2)
df_neg_wide['Ontology'] = df_neg_wide['Ontology'].str.replace(r'-', '_')

In [6]:
column_path = 'Data/model_data/neg/negmodelcolumn_all.pkl'
replacement_dict_path = 'Data/model_data/neg/neg_replacement_dict_all.pkl'
model_path = "models/neg_class_model"

pred1st, predcandidate, num, df_test_predclass = test_base.cal_accscore(df_neg_wide, replacement_dict_path, column_path, model_path)

250/250 [==============================] - 1s 5ms/step


In [7]:
pred1st1, predcandidate1, num1, df_test_predclass1 = test_base.make_pred_summary(df_neg_wide, replacement_dict_path, column_path, model_path, (1, 11), type='btw', title = 'test Accuracy in Different curator')
pred1st2, predcandidate2, num2, df_test_predclass2 = test_base.make_pred_summary(df_neg_wide, replacement_dict_path, column_path, model_path, [78],  title = 'test Accuracy in Waters')
pred1st3, predcandidate3, num3, df_test_predclass3 = test_base.make_pred_summary(df_neg_wide, replacement_dict_path, column_path, model_path, [79],  title = 'test Accuracy in Thermo')
pred1st4, predcandidate4, num4, df_test_predclass4 = test_base.make_pred_summary(df_neg_wide, replacement_dict_path, column_path, model_path, [80],  title = 'test Accuracy in Agilent')
pred1st5, predcandidate5, num5, df_test_predclass5 = test_base.make_pred_summary(df_neg_wide, replacement_dict_path, column_path, model_path, [83, 84],  title = 'test Accuracy in Bruker')
pred1st6, predcandidate6, num6, df_test_predclass6 = test_base.make_pred_summary(df_neg_wide, replacement_dict_path, column_path, model_path, [81],  title = 'test Accuracy in Sciex 6600-SWATH')

test Accuracy in Different curator
119/119 [==============================] - 1s 5ms/step
1st pred : 3056
candidate : 3394
total: 3790
----
test Accuracy in Waters
2/2 [==============================] - 0s 9ms/step
1st pred : 13
candidate : 34
total: 46
----
test Accuracy in Thermo
4/4 [==============================] - 0s 5ms/step
1st pred : 95
candidate : 96
total: 98
----
test Accuracy in Agilent
9/9 [==============================] - 0s 6ms/step
1st pred : 186
candidate : 248
total: 265
----
test Accuracy in Bruker
7/7 [==============================] - 0s 6ms/step
1st pred : 151
candidate : 191
total: 195
----
test Accuracy in Sciex 6600-SWATH
3/3 [==============================] - 0s 6ms/step
1st pred : 66
candidate : 74
total: 77
----


In [10]:
data = [['Different curator(Sciex TripleTOF 6600)', num1, pred1st1, predcandidate1-pred1st1],
        ['Waters XevoG2 QTOF', num2, pred1st2, predcandidate2-pred1st2],
        ['Thermo Q Exactive Plus', num3, pred1st3, predcandidate3-pred1st3],
        ['Agilent 6546 QTOF', num4, pred1st4, predcandidate4-pred1st4],
        ['Bruker timsTOF Pro', num5, pred1st5, predcandidate5-pred1st5],
        ['Sciex TripleTOF 6600(SWATH)', num6, pred1st6, predcandidate6-pred1st6]]

df_predresult = pd.DataFrame(data, columns=['Name', 'Sample number', '1st pred', 'Candidate'])
df_predresult['Non predict'] = df_predresult['Sample number']-(df_predresult['1st pred']+df_predresult['Candidate'])

df_predresult.to_csv('Data/test_data/neg/rikenlipid_df_neg_model_pred_result.csv', index=False)